In [20]:
# load packages

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
from datetime import datetime
from dateutil.relativedelta import relativedelta
from danish_banking_holidays.calendar import DanishBankingCalendar
from pyspark.sql.types import BooleanType, StringType
from pyspark.sql.functions import udf

spark = SparkSession.builder.appName("DanishBankingHolidayApp").getOrCreate() # type: ignore
# Create a UDF that wraps the calendar.is_holiday method

# Initialize the calendar - REQUIRED!
calendar = DanishBankingCalendar()

@udf(returnType=BooleanType())
def is_danish_holiday(date):
    return calendar.is_holiday(date)

@udf(returnType=StringType())
def holiday_name(date):
    calendar_dict = calendar.get_holiday_name(date)
    if not calendar_dict:
        return None
    return calendar_dict.get(date)

# Register the UDF so it can be used in SQL expressions
spark.udf.register("is_danish_holiday", is_danish_holiday)
spark.udf.register("holiday_name", holiday_name)


# define boundaries
startdate = (datetime.now() + relativedelta(years=-2)).replace(month=1, day=1) 
enddate   = (datetime.now() + relativedelta(years=2)).replace(month=12, day=31)  # datetime.strptime('2023-10-01','%Y-%m-%d')


# define column names and its transformation rules on the Date column
column_rule_df = spark.createDataFrame([
    ("DateID",              "cast(date_format(date, 'yyyyMMdd') as int)"),     # 20230101
    ("Year",                "year(date)"),                                     # 2023
    ("Quarter",             "quarter(date)"),                                  # 1
    ("Month",               "month(date)"),                                    # 1
    ("Day",                 "day(date)"),                                      # 1
    ("Week",                "weekofyear(date)"),                               # 1
    ("QuarterNameLong",     "date_format(date, 'QQQQ')"),                      # 1st qaurter
    ("QuarterNameShort",    "date_format(date, 'QQQ')"),                       # Q1
    ("QuarterNumberString", "date_format(date, 'QQ')"),                        # 01
    ("MonthNameLong",       "date_format(date, 'MMMM')"),                      # January
    ("MonthNameShort",      "date_format(date, 'MMM')"),                       # Jan
    ("MonthNumberString",   "date_format(date, 'MM')"),                        # 01
    ("DayNumberString",     "date_format(date, 'dd')"),                        # 01
    ("WeekNameLong",        "concat('week', lpad(weekofyear(date), 2, '0'))"), # week 01
    ("WeekNameShort",       "concat('w', lpad(weekofyear(date), 2, '0'))"),    # w01
    ("WeekNumberString",    "lpad(weekofyear(date), 2, '0')"),                 # 01
    ("DayOfWeek",           "dayofweek(date)"),                                # 1
    ("YearMonthString",     "date_format(date, 'yyyy/MM')"),                   # 2023/01
    ("DayOfWeekNameLong",   "date_format(date, 'EEEE')"),                      # Sunday
    ("DayOfWeekNameShort",  "date_format(date, 'EEE')"),                       # Sun
    ("DayOfMonth",          "cast(date_format(date, 'd') as int)"),            # 1
    ("DayOfYear",           "cast(date_format(date, 'D') as int)"),            # 1
    ("MonthNameLong_DK",   "CASE WHEN month(date) = 1 THEN 'Januar' WHEN month(date) = 2 THEN 'Februar' WHEN month(date) = 3 THEN 'Marts' WHEN month(date) = 4 THEN 'April' WHEN month(date) = 5 THEN 'Maj' WHEN month(date) = 6 THEN 'Juni' WHEN month(date) = 7 THEN 'Juli' WHEN month(date) = 8 THEN 'August' WHEN month(date) = 9 THEN 'September' WHEN month(date) = 10 THEN 'Oktober' WHEN month(date) = 11 THEN 'November' WHEN month(date) = 12 THEN 'December' ELSE '' END"), # Januar
    ("MonthNameShort_DK",  "CASE WHEN month(date) = 1 THEN 'Jan' WHEN month(date) = 2 THEN 'Feb' WHEN month(date) = 3 THEN 'Mar' WHEN month(date) = 4 THEN 'Apr' WHEN month(date) = 5 THEN 'Maj' WHEN month(date) = 6 THEN 'Jun' WHEN month(date) = 7 THEN 'Jul' WHEN month(date) = 8 THEN 'Aug' WHEN month(date) = 9 THEN 'Sep' WHEN month(date) = 10 THEN 'Okt' WHEN month(date) = 11 THEN 'Nov' WHEN month(date) = 12 THEN 'Dec' ELSE '' END"),                                    # Jan
    ("DayofWeekNameLong_DK", "CASE WHEN dayofweek(date) = 1 THEN 'Søndag' WHEN dayofweek(date) = 2 THEN 'Mandag' WHEN dayofweek(date) = 3 THEN 'Tirsdag' WHEN dayofweek(date) = 4 THEN 'Onsdag' WHEN dayofweek(date) = 5 THEN 'Torsdag' WHEN dayofweek(date) = 6 THEN 'Fredag' WHEN dayofweek(date) = 7 THEN 'Lørdag' ELSE '' END") ,                                                                                                                                         # Sonntag
    ("IsHoliday", "cast(is_danish_holiday(date) as int)"),  # 1 if holiday else 0
    ("IsToady",   "CASE WHEN date = current_date() THEN 1 ELSE 0 END") , # 1 if today else 0
    ("HolidayName", "holiday_name(date)")  # name of the holiday or null
], ["new_column_name", "expression"])



# explode dates between the defined boundaries into one column
start = int(startdate.timestamp())
stop  = int(enddate.timestamp())
df = spark.range(start, stop, 60*60*24).select(col("id").cast("timestamp").cast("date").alias("Date"))

# this loops over all rules defined in column_rule_df adding the new columns
for row in column_rule_df.collect():
    new_column_name = row["new_column_name"]
    expression = expr(row["expression"])
    df = df.withColumn(new_column_name, expression)

df.show(5)
# df.orderBy(col("Date").desc()).show(5)
# COMMAND ----------
df.select("Date", "IsHoliday", "DayOfWeekNameLong_DK", "HolidayName").filter((col("IsHoliday") == 1) & (col("Year") == 2025) & (col("DayOfWeek") != 1) & (col("DayOfWeek") != 7)).orderBy(col("Date")).show(100)
df.createOrReplaceTempView("dates")
spark.sql("SELECT * FROM dates WHERE isToady = 1").show(20)
# display(df.withColumn("Playground", expr("date_format(date, 'yyyyMMDD')")))


ConnectionRefusedError: [Errno 61] Connection refused